1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их, как делалось на вебинаре
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

In [1]:
import pandas as pd
import numpy as np


# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.utils import  get_similar_items_recommendation, get_similar_users_recommendation, get_top_N_items

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
PATH = 'C:/Users/ASER/Desktop/GeekBrains/RecSys/data/'

In [4]:
data = pd.read_csv(PATH + 'transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
item_features = pd.read_csv(PATH + 'product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [7]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 90386 to 3720


In [8]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробовать другие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,818981,819255,819308,819330,819594,819765,819840,819845,819927,819978,...,15926887,15926927,15972074,15972298,16100266,16729299,16729415,16770156,16809649,17104444
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [10]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице ! 

In [11]:
%%time
model = AlternatingLeastSquares(factors=20, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=-1)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


Wall time: 4.51 s


-----

In [12]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False,                                     
                                    filter_items=[itemid_to_id[999999]],
                                    recalculate_user=True)]
    
    return res

In [13]:
%%time
    
result['bm25'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['bm25'], row['actual']), axis=1).mean()

Wall time: 24.6 s


0.045203415369161226

In [14]:
def get_top_N_items(cond_1, N):
    cond_2 = data_train['item_id'] != 999999
    
    popularity = data_train[cond_1 & cond_2].\
                 groupby('item_id')['quantity'].count().reset_index()
    popularity.sort_values('quantity', ascending=False, inplace=True)
    popularity = popularity.head(N)
    top_N_items = popularity['item_id'].tolist()
    
    return top_N_items


def get_similar_items_recommendation(user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

    cond = data_train['user_id'] == user    
    top_N_items = get_top_N_items(cond, N)

    def get_rec(model, x):
        recs = model.similar_items(itemid_to_id[x], N=2)[1][0]    
        return id_to_itemid[recs]
    
    res = [get_rec(model, i) for i in top_N_items]
    
    return res


def get_similar_users_recommendation(user, model, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    recs = model.similar_users(userid_to_id[user], N+1)[1:]
    similar_users_list = [id_to_userid[j] for j in [i[0] for i in recs]]    
    
    cond = data_train['user_id'].isin(similar_users_list)
    res = get_top_N_items(cond, N)
    
    return res

In [15]:
user = 2494
get_similar_items_recommendation(user, model)

[1135476, 1094308, 823704, 1072799, 912704]

In [16]:
get_similar_users_recommendation(user, model)

[1126899, 908318, 993638, 1054539, 1034955]

При генерировании рекомендаций товаров, похожих на топ-N купленных пользователем товаров, для большого числа пользователей, лучше воспользоваться функцией:

In [17]:
def get_similar_items_recommendation_all(result, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
    popularity = data_train.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
    popularity.sort_values('quantity', ascending=False, inplace=True)
    popularity = popularity[popularity['item_id'] != 999999]
    popularity = popularity.groupby('user_id').head(N)
    
    def get_rec(model, x):
        recs = model.similar_items(itemid_to_id[x], N=2)[1][0]    
        return id_to_itemid[recs]
    
    popularity['similar_items_recs'] = popularity['item_id'].\
                                             apply(lambda x: get_rec(model, x))
    recommendations_similar_items = popularity.groupby('user_id')['similar_items_recs'].\
                                             unique().reset_index()
    
    # Дополним недостающие рекомендации популярными товарами:
    to_add_list = popularity['item_id'][:5].tolist()

    def add_rec(x, to_add_list):
        additions = [i for i in to_add_list if i not in x]
        return [*x, *additions][:N]

    recommendations_similar_items['similar_items_recs'] =\
    recommendations_similar_items['similar_items_recs'].apply(lambda x: add_rec(x, to_add_list))

    result = result.merge(recommendations_similar_items, on='user_id', how='left')

    ind = result[result['similar_items_recs'].isnull()].index.tolist()
    result.loc[ind, 'similar_items_recs'] = str(to_add_list)
    
    return result

In [18]:
result = get_similar_items_recommendation_all(result, model)
result.apply(lambda row: precision_at_k(row['similar_items_recs'], row['actual']), axis=1).mean()

0.03867403314917127

In [19]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """
    
    def __init__(self, data, weighting=True):
        self.total_items_rating = self.get_total_items_rating(data)
        self.each_user_items_rating = self.get_each_user_items_rating(data)
        
        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)
        
        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T 
        
        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)
        
     
    @staticmethod
    def prepare_matrix(data):
        user_item_matrix = pd.pivot_table(data, 
                                          index='user_id', columns='item_id', 
                                          values='quantity', # Можно пробовать другие варианты
                                          aggfunc='count', 
                                          fill_value=0
                                         )
        user_item_matrix = user_item_matrix.astype(float)        
        
        return user_item_matrix
    
    
    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))
        
        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id
    
     
    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
    
        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return own_recommender
    
    
    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""
        
        model = AlternatingLeastSquares(factors=n_factors, 
                                             regularization=regularization,
                                             iterations=iterations,  
                                             num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())
        
        return model
    
    
    @staticmethod
    def get_total_items_rating(data):
        popularity = (data.groupby('item_id')['user_id'].nunique()\
                                     / data.user_id.nunique()).reset_index()
        popularity = popularity[popularity['item_id'] != 999999]
        popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)
        popularity.sort_values('share_unique_users', ascending=False, inplace=True)

        return popularity['item_id'].tolist()
    
    
    @staticmethod
    def get_each_user_items_rating(data):
        popularity = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity = popularity[popularity['item_id'] != 999999]
        popularity.sort_values('quantity', ascending=False, inplace=True)        
        
        return popularity
 
     
    def get_similar_items_recommendation(self, user, N=5):
        """
        Рекомендуем товары, похожие на топ-N купленных юзером товаров
        """        
               
        df = self.each_user_items_rating    
        user_top_N_items = df[df['user_id'] == user].item_id.tolist()[:N]
        res = []
        for i in user_top_N_items:
            recs = self.model.similar_items(self.itemid_to_id[i], N=2)[1][0]
            item_id = self.id_to_itemid[recs]
            res.append(item_id)
        
        # Дополним недостающие рекомендации популярными товарами:
        additions = [i for i in self.total_items_rating[:N] if i not in res]
        res = [*res, *additions][:N]

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res
    
    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
        recs = self.model.similar_users(self.userid_to_id[user], N+1)[1:]
        similar_users_list = [self.id_to_userid[j] for j in [i[0] for i in recs]]
        res = []
        for i in similar_users_list:
            recs = model.recommend(userid=userid_to_id[i], 
                                   user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                                   N=1, 
                                   filter_already_liked_items=False, 
                                   filter_items=[itemid_to_id[999999]],
                                   recalculate_user=True)[0]
            item_id = id_to_itemid[recs[0]] 
            res.append(item_id)
        
        # Дополним недостающие рекомендации популярными товарами:
        additions = [i for i in self.total_items_rating[:N] if i not in res]
        res = [*res, *additions][:N] 
        
        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

----

Проверка, что все работает

In [21]:
#%load_ext autoreload
#%autoreload 
#from src.utils import  get_similar_items_recommendation, get_similar_users_recommendation, get_top_N_items

In [22]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [23]:
recomender = MainRecommender(data_train)

In [24]:
user_id = 58
recomender.get_similar_items_recommendation(user_id)

[1033845, 1047769, 901211, 5565925, 8181451]

In [25]:
recomender.get_similar_users_recommendation(user_id)

[9836460, 866871, 8014645, 6548453, 930118]